<a href="https://colab.research.google.com/github/jlonge4/AppointmentSchedulingGUI/blob/main/faiss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -qU langchain openai llama-index faiss-cpu sentence_transformers chromadb

In [ ]:
# from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
import os
import openai

llm = ChatOpenAI(
    openai_api_key=key,
    temperature=0,
    model_name="gpt-3.5-turbo"
)

In [100]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


In [ ]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')
# create sentence embeddings


In [15]:
from langchain.schema import Document
from langchain.document_loaders import WebBaseLoader

two_web_links = ["https://www.databricks.com/","https://help.databricks.com","https://databricks.com/try-databricks","https://help.databricks.com/s/","https://docs.databricks.com","https://kb.databricks.com/","http://docs.databricks.com/getting-started/index.html","http://docs.databricks.com/introduction/index.html","http://docs.databricks.com/getting-started/tutorials/index.html","http://docs.databricks.com/release-notes/index.html","http://docs.databricks.com/ingestion/index.html","http://docs.databricks.com/exploratory-data-analysis/index.html","http://docs.databricks.com/data-preparation/index.html","http://docs.databricks.com/data-sharing/index.html","http://docs.databricks.com/marketplace/index.html","http://docs.databricks.com/workspace-index.html","http://docs.databricks.com/machine-learning/index.html","http://docs.databricks.com/sql/index.html","http://docs.databricks.com/delta/index.html","http://docs.databricks.com/dev-tools/index.html","http://docs.databricks.com/integrations/index.html","http://docs.databricks.com/administration-guide/index.html","http://docs.databricks.com/security/index.html","http://docs.databricks.com/data-governance/index.html","http://docs.databricks.com/lakehouse-architecture/index.html","http://docs.databricks.com/reference/api.html","http://docs.databricks.com/resources/index.html","http://docs.databricks.com/whats-coming.html","http://docs.databricks.com/archive/index.html","http://docs.databricks.com/lakehouse/index.html","http://docs.databricks.com/getting-started/quick-start.html","http://docs.databricks.com/getting-started/etl-quick-start.html","http://docs.databricks.com/getting-started/lakehouse-e2e.html","http://docs.databricks.com/getting-started/free-training.html","http://docs.databricks.com/sql/language-manual/index.html","http://docs.databricks.com/error-messages/index.html","http://www.apache.org/","https://databricks.com/privacy-policy","https://databricks.com/terms-of-use"]
web_links = ["https://www.pinecone.io/learn/vector-embeddings/", "https://www.featureform.com/post/the-definitive-guide-to-embeddings"]
loader = WebBaseLoader(web_links)
documents = loader.load()

In [28]:
from importlib.metadata import metadata
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

In [45]:
docs = []
for d in all_splits:
  docs.append(d.page_content)

In [ ]:
docs[0]

INDEXFLATL2

In [103]:
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain

vectorstore = FAISS.from_documents(all_splits, embedding_function)

In [94]:
import time
query = 'What does pinecone define as embeddings'

chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)

chat_history = []
start = time.time()
result = chain({"question": query, "chat_history": chat_history})
end = time.time()
print(end - start)

3.9789793491363525


In [91]:
L2 = result['answer']

INDEXIVFFLAT

In [52]:
sentence_embeddings = model.encode(docs)
d = sentence_embeddings.shape[1]
d

768

In [56]:
import faiss

nlist = 16  # how many cells
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer,d, nlist)

In [58]:
index.train(sentence_embeddings)
index.add(sentence_embeddings)

In [108]:
def get_ivf():
  k = 4
  xq = model.encode([query])
  D, I = index.search(xq, k)
  print(I)
  return I[0][0]


In [109]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["source"],
    template="Given the context ### {source}### What does pinecone define as embeddings?",
)

from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)
start = time.time()

# Run the chain only specifying the input variable.

IVFFLAT = chain.run(docs[get_ivf()])
print(IVFFLAT)
end = time.time()
print(end - start)

[[16 48  5 37]]


Embeddings are numerical representations of words, phrases, and other elements of the text. These representations are derived from a machine learning model such as Word2Vec or BERT that "learns" the relationships between words from large corpuses of text and then assigns a vector to each word. Embeddings capture the semantic relationship between words by considering how they are used in the context of the text.
3.01558780670166


INDEXIVFPQ

In [78]:
m = 4  # number of centroid IDs in final compressed vectors
bits = 4 # number of bits in each centroid

quantizer = faiss.IndexFlatL2(d)  # we keep the same L2 distance flat index
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits)

In [82]:
index.train(sentence_embeddings)
index.add(sentence_embeddings)
index.nprobe = 10  # align to previous IndexIVFFlat nprobe value

In [110]:
def get_ivfpq():
  D, I = index.search(xq, k)
  print(I)
  return I[0][0]

In [111]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["source"],
    template="Given the context ### {source}### What does pinecone define as embeddings?",
)

from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)
start = time.time()

# Run the chain only specifying the input variable.
result = chain.run(docs[get_ivfpq()])
print(result)
IVFPQ = result
end = time.time()
print(end - start)

[[16 48  5 37]]


Pinecone defines embeddings as numerical representations of text that capture the semantic relationships among words in the text. Embeddings are used to represent word meaning and can be used to help computers understand the context of words in a sentence.
1.8402493000030518


In [112]:
times = {"INDEXFLATL2": 3.978,
 "INDEXIVFFLAT": 3.015,
 "INDEXIVFPQ": 1.840}
from pprint import pprint
pprint(str(times).strip('{').strip(',').strip('}'))
answers = times = {"INDEXFLATL2": L2,
 "INDEXIVFFLAT": IVFFLAT.strip('\n'),
 "INDEXIVFPQ": IVFPQ.strip('\n')}
print('\n')
pprint(answers)

"'INDEXFLATL2': 3.978, 'INDEXIVFFLAT': 3.015, 'INDEXIVFPQ': 1.84"


{'INDEXFLATL2': ' Pinecone does not define embeddings. Embeddings are a '
                'concept used in machine learning and natural language '
                'processing, and they are central to many algorithms. '
                'Embeddings are used in applications like recommendation '
                'engines, voice assistants, language translators, and more.',
 'INDEXIVFFLAT': 'Embeddings are numerical representations of words, phrases, '
                 'and other elements of the text. These representations are '
                 'derived from a machine learning model such as Word2Vec or '
                 'BERT that "learns" the relationships between words from '
                 'large corpuses of text and then assigns a vector to each '
                 'word. Embeddings capture the semantic relationship between '
                 'words by considering how they are used in the context of the '
           